# Westeros Tutorial - Introducing emissions (part1/2): Adding emission bounds

In the first part, the model chose not to base the power system on wind power as electricity from wind turbines was more expensive than electricity produced from coal. However, we now introduce emissions to investigate the impact of climate policy. Let's see what happens then.

**Pre-requisites**
- You have the *MESSAGEix* framework installed and working
- You have run Westeros baseline scenario (``westeros_baseline.ipynb``) and solved it successfully

In [ ]:
import ixmp

mp = ixmp.Platform()

## Making a clone of the existing scenario 'baseline'

In [ ]:
import message_ix

model = "Westeros Electrified"

base = message_ix.Scenario(mp, model=model, scenario="baseline")
scen = base.clone(
    model,
    "emission_bound",
    "introducing an upper bound on emissions",
    keep_solution=False,
)
scen.check_out()

In [ ]:
year_df = scen.vintage_and_active_years()
vintage_years, act_years = year_df["year_vtg"], year_df["year_act"]
model_horizon = scen.set("year")
country = "Westeros"

## Introducing Emissions

In [ ]:
from message_ix import make_df

# First we introduce the emission of CO2 and the emission category GHG
scen.add_set("emission", "CO2")
scen.add_cat("emission", "GHG", "CO2")

# Then we add new units to the model library (needed only once)
mp.add_unit("tCO2/kWa")
mp.add_unit("MtCO2")

# Last we add CO2 emissions to the coal powerplant
emission_factor = make_df(
    "emission_factor",
    node_loc=country,
    year_vtg=vintage_years,
    year_act=act_years,
    mode="standard",
    unit="tCO2/kWa",
    technology="coal_ppl",
    emission="CO2",
    value=7.4,
)
scen.add_par("emission_factor", emission_factor)

## Define a Bound on Emissions

The `"type_year": "cumulative"` assigns an upper bound on the *weighted average of emissions* over the entire time horizon.

In [ ]:
scen.add_par(
    "bound_emission",
    make_df(
        "bound_emission",
        node=country,
        type_emission="GHG",
        type_tec="all",
        type_year="cumulative",
        value=500.0,
        unit="MtCO2",
    ),
)

## Time to Solve the Model

In [ ]:
scen.commit(comment="Introducing emissions and setting an upper bound")
scen.set_as_default()

In [ ]:
scen.solve()

In [ ]:
scen.var("OBJ")["lvl"]

## Plotting Results

In [ ]:
from message_ix.report import Reporter
from message_ix.util.tutorial import prepare_plots

%matplotlib inline

rep = Reporter.from_scenario(scen)
prepare_plots(rep)

### Activity

How much energy is generated in each time period from the different potential sources?

In [ ]:
rep.set_filters(t=["coal_ppl", "wind_ppl"])
rep.get("plot activity")

### Capacity

How much capacity of each plant is installed in each period?

In [ ]:
rep.get("plot capacity")

### Electricity Price

And how much does the electricity cost? These prices are in fact **shadow prices** taken from the **dual variables** of the model solution. They reflect the marginal cost of electricity generation (i.e., the additional cost of the system for supplying one more unit of electricity), which is in fact the marginal cost of the most expensive generator.  

Notice the drop in the price in the period of 710? 

The share of the activity of the coal power plant is higher in the period of 710 than in the period of 700. With lower electricity costs of the coal power plant, this leads to reduced electricity costs. To stay within the emission bounds with an increased electricity demand, in the upcoming period of 720, the capacity of the wind turbines increases drastically, while the capacity of the coal power plants remains the same. This results in the increased electricity price.

In [ ]:
rep.set_filters(t=None, c=["light"])
rep.get("plot prices")

## Close the connection to the database

In [ ]:
mp.close_db()